In [ ]:
#Basic Info
# 198 different stocks to measure
# incrementing from 0 seconds to 540 seconds by 10s increments over a 480 day period
# target value = wap
#

In [1]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('train.csv')

In [3]:
#Limit data size... for local runtime only
df = df.sample(n=10000)

In [4]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
5196809,29,477,140,518796.47,-1,0.999631,1.690584e+07,NaN,NaN,0.999599,14977.28,1.000314,21545.25,0.999892,-4.479885,26249,477_140_29
2900959,61,268,220,375340.94,1,1.000619,1.793787e+06,NaN,NaN,1.000303,22586.25,1.000934,72163.00,1.000454,1.840591,14762,268_220_61
4494894,114,413,240,3819122.82,1,1.000538,7.698609e+06,NaN,NaN,1.000289,46539.20,1.000837,22479.52,1.000659,12.149811,22739,413_240_114
404697,35,38,180,15692751.36,1,0.999914,6.246388e+06,NaN,NaN,0.999742,13122.69,1.000000,34848.00,0.999812,-4.619956,2108,38_180_35
4972863,83,456,490,9760321.19,1,1.000841,2.367051e+08,1.001877,1.001531,1.000496,380884.14,1.000841,611196.17,1.000628,-0.960231,25129,456_490_83


In [5]:
X = df[['bid_price', 'bid_size', 'ask_price','ask_size', 'near_price', 'matched_size']]

In [6]:
Y = df['target']

In [7]:
#Linear Regression Model Imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader, TensorDataset

In [8]:
Y = np.array(Y)
X = np.array(X)

In [9]:
Y = Y.reshape(-1,1)

In [10]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [11]:
#if null value, replces it with mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_scaled)
Y_imputed = imputer.fit_transform(Y)

In [12]:
X_tensor = torch.tensor(X_imputed, dtype=torch.float32)
Y_tensor = torch.tensor(Y_imputed, dtype=torch.float32)
sequence_length = 10  # You can adjust this value based on your preference
X_tensor = X_tensor.view(-1, sequence_length, X.shape[1])
Y_tensor = Y_tensor.view(-1, sequence_length, Y.shape[1])
print(X_tensor.shape)
print(Y_tensor.shape)
print(X_imputed.shape)
print(Y_imputed.shape)

torch.Size([1000, 10, 6])
torch.Size([1000, 10, 1])
(10000, 6)
(10000, 1)


In [13]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [14]:
input_size = X.shape[1]
hidden_size = 50  # You can adjust this value based on your preference
output_size = 1
model = LSTMModel(input_size, hidden_size, output_size)


In [15]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [31]:
#figure out Kaggle Kernels or other GPU
num_epochs = 10
for epoch in range(num_epochs):
    outputs = model(X_tensor)
    optimizer.zero_grad()
    loss = criterion(outputs, Y_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


RuntimeError: [enforce fail at alloc_cpu.cpp:80] data. DefaultCPUAllocator: not enough memory: you tried to allocate 25058516992 bytes.

In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(X_tensor)

In [ ]:
predictions_rescaled = scaler.inverse_transform(predictions.numpy().reshape(-1, 1))
Y_rescaled = scaler.inverse_transform(Y.values.reshape(-1, 1))

In [ ]:
mse = mean_squared_error(Y_rescaled, predictions_rescaled)
print(f'Mean Squared Error: {mse}')

In [16]:
#Linear Regression Model Prediction
X_train, X_test, Y_train, Y_test = train_test_split(X_imputed, Y_imputed, test_size=0., random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, Y_train)

Y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(Y_test, Y_pred)
me = mean_absolute_error(Y_test, Y_pred)
print("predicted:", Y_pred)
print("Actual: ", Y_test)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {me}')